# Extract Transform Load Project (ETL)

### Extracting Original data sources

In [1]:
import os
import utm
import pandas as pd
import matplotlib.pyplot as plt
import sqlalchemy
from sqlalchemy import create_engine
import reverse_geocoder as rg
import pprint

In [2]:
csv_file = "./Data Files/CWD_Surveillance_data_2000_2010.csv"
cwd_2000_2010_df = pd.read_csv(csv_file)
cwd_2000_2010_df.head()

,Case_ID,Province,Harvest Type,ObsDate,Month_Obs,Year_Obs,Easting,Northing,Species,Sex,CWD Status
0,100,Saskatchewan,Hunter,Undetermined,Undetermined,2000,580000.0,5850000.0,Mule Deer,Male,Positive
1,101,Saskatchewan,Hunter,Undetermined,Undetermined,2001,580000.0,5830000.0,Mule Deer,Male,Positive
2,102,Saskatchewan,Hunter,Undetermined,Undetermined,2002,570000.0,5830000.0,Mule Deer,Male,Positive
3,103,Saskatchewan,Hunter,Undetermined,Undetermined,2003,610000.0,5950000.0,Mule Deer,Female,Positive
4,104,Saskatchewan,Hunter,Undetermined,Undetermined,2004,590000.0,5840000.0,Mule Deer,Male,Positive


In [3]:
cwd_2000_2010_df['Province'].value_counts()

Alberta         16330
Saskatchewan     3222
Name: Province, dtype: int64

In [4]:
csv_file = "./Data Files/CWD_surveillance_validation_data_2011_2013.csv"
cwd_2011_2013_df = pd.read_csv(csv_file)
cwd_2011_2013_df.head()

,Case,Year,Easting,Northing,CWD_Status,Species,Sex
0,10000,2011,432845,5849162,Negative,White-tailed Deer,Male
1,10001,2011,429500,5868700,Negative,White-tailed Deer,Male
2,10002,2011,438723,5896022,Negative,White-tailed Deer,Male
3,10003,2011,428623,5862820,Negative,Mule Deer,Male
4,10004,2011,431230,5882820,Negative,Mule Deer,Female


In [6]:
csv_file = "./Data Files/Census_Data.csv"
census_df = pd.read_csv(csv_file)
census_df

,Province,Population
0,Newfoundland and Labrador,"155,304.00"
1,Prince Edward Island,"57,887.00"
2,Nova Scotia,"512,612.00"
3,New Brunswick,"195,361.00"
4,Quebec,"3,064,452.00"
5,Ontario,"5,255,754.00"
6,Manitoba,"723,747.00"
7,Saskatchewan,"450,418.00"
8,Alberta,"2,001,524.00"
9,Yukon,"23,276.00"


# Transform: What data cleaning or transformation was required.

In [8]:
#creating a new cleaned up dataframe so the two years will match and be appended
new_cwd_2000_2010_df = cwd_2000_2010_df[['Case_ID', 'Province', 'Year_Obs', 'Species', 'Sex', 'CWD Status']].copy()
new_cwd_2000_2010_df.head()

,Case_ID,Province,Year_Obs,Species,Sex,CWD Status
0,100,Saskatchewan,2000,Mule Deer,Male,Positive
1,101,Saskatchewan,2001,Mule Deer,Male,Positive
2,102,Saskatchewan,2002,Mule Deer,Male,Positive
3,103,Saskatchewan,2003,Mule Deer,Female,Positive
4,104,Saskatchewan,2004,Mule Deer,Male,Positive


In [11]:
#Renaming columns so headers will match for append. Year is keyword in SQL so had to rename column to load.
new_cwd_2000_2010_df = new_cwd_2000_2010_df.rename(columns={"Year_Obs":"Years", "CWD Status":"CWD_Status"})
new_cwd_2000_2010_df.head()

,Case_ID,Province,Year,Species,Sex,CWD_Status
0,100,Saskatchewan,2000,Mule Deer,Male,Positive
1,101,Saskatchewan,2001,Mule Deer,Male,Positive
2,102,Saskatchewan,2002,Mule Deer,Male,Positive
3,103,Saskatchewan,2003,Mule Deer,Female,Positive
4,104,Saskatchewan,2004,Mule Deer,Male,Positive


In [10]:
#Renaming columns so headers will match for append. Year is keyword in SQL so had to rename column to load.
cwd_2011_2013_df = cwd_2011_2013_df.rename(columns={"Case":"Case_ID", "Year": "Years"})
cwd_2011_2013_df.head()

,Case_ID,Year,Easting,Northing,CWD_Status,Species,Sex
0,10000,2011,432845,5849162,Negative,White-tailed Deer,Male
1,10001,2011,429500,5868700,Negative,White-tailed Deer,Male
2,10002,2011,438723,5896022,Negative,White-tailed Deer,Male
3,10003,2011,428623,5862820,Negative,Mule Deer,Male
4,10004,2011,431230,5882820,Negative,Mule Deer,Female


In [9]:
#Test of utm coordinates to lat lon
utm.to_latlon(548750.7613, 5826273.207, 12, "N")

(52.584309997959735, -110.28045998665556)

In [12]:
#Taking UTM x y coordinates with zone 12 N and converting it into LAT LON then converting that into the missing province data.
easting = []
northing = []
latitude = []
longitude = []
address = []
province =[]

easting = cwd_2011_2013_df["Easting"]
northing = cwd_2011_2013_df["Northing"]

for i in range(0, len(easting)):
   latlong = utm.to_latlon(easting[i], northing[i], 12, "N")


   def reverseGeocode(latlong):
     result = rg.search(latlong)
     address.append(result)
    
   if __name__=="__main__":
     reverseGeocode(latlong)

   province.append(address[i][0]["admin1"])

Loading formatted geocoded file...


In [13]:
#Creating our last missing piece of data as a new column
cwd_2011_2013_df['Province']= province

cwd_2011_2013_df

,Case_ID,Year,Easting,Northing,CWD_Status,Species,Sex,Province
0,10000,2011,432845,5849162,Negative,White-tailed Deer,Male,Alberta
1,10001,2011,429500,5868700,Negative,White-tailed Deer,Male,Alberta
2,10002,2011,438723,5896022,Negative,White-tailed Deer,Male,Alberta
3,10003,2011,428623,5862820,Negative,Mule Deer,Male,Alberta
4,10004,2011,431230,5882820,Negative,Mule Deer,Female,Alberta
5,10005,2011,444623,5897822,Negative,Mule Deer,Female,Alberta
6,10006,2011,428592,5872368,Negative,Mule Deer,Female,Alberta
7,10007,2011,445655,5870519,Negative,White-tailed Deer,Male,Alberta
8,10008,2011,430831,5881672,Negative,White-tailed Deer,Male,Alberta
9,10009,2011,430200,5867300,Negative,White-tailed Deer,Male,Alberta


In [14]:
#Saving our data from above into csv because the for loop takes a while to run on certain machines.
cwd_2011_2013_df.to_csv('./Output/New_2011_2013.csv')

In [ ]:
#Usable once the for loop has been run once so that you don't need to run that step again to continue the transformation.

# csv_file = "./Output/New_2011_2013.csv"
# new_cwd_2011_2013_df = pd.read_csv(csv_file)
# new_cwd_2011_2013_df



In [15]:
#No longer need these columns and must remove to append.
new_cwd_2011_2013_df = cwd_2011_2013_df.drop(['Easting', 'Northing'], axis=1)
new_cwd_2011_2013_df.head()

,Case_ID,Year,CWD_Status,Species,Sex,Province
0,10000,2011,Negative,White-tailed Deer,Male,Alberta
1,10001,2011,Negative,White-tailed Deer,Male,Alberta
2,10002,2011,Negative,White-tailed Deer,Male,Alberta
3,10003,2011,Negative,Mule Deer,Male,Alberta
4,10004,2011,Negative,Mule Deer,Female,Alberta


In [16]:
#Appending the two DataFrames together into one so the new transformed file is ready to load into SQL.
cwd_2000_2013_df = new_cwd_2000_2010_df.append(new_cwd_2011_2013_df)
cwd_2000_2013_df.head()

C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\frame.py:6211: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  sort=sort)


,CWD_Status,Case_ID,Province,Sex,Species,Year
0,Positive,100,Saskatchewan,Male,Mule Deer,2000
1,Positive,101,Saskatchewan,Male,Mule Deer,2001
2,Positive,102,Saskatchewan,Male,Mule Deer,2002
3,Positive,103,Saskatchewan,Female,Mule Deer,2003
4,Positive,104,Saskatchewan,Male,Mule Deer,2004


In [17]:
#Checking Data
cwd_2000_2013_df.tail()

,CWD_Status,Case_ID,Province,Sex,Species,Year
9323,Positive,19323,Alberta,Male,Mule Deer,2011
9324,Positive,19324,Alberta,Male,Mule Deer,2011
9325,Positive,19325,Alberta,Male,Mule Deer,2011
9326,Positive,19326,Alberta,Female,Mule Deer,2011
9327,Positive,19327,Alberta,Male,Mule Deer,2011


In [18]:
#Checking Data
cwd_2000_2013_df['Year'].value_counts()

2008    5653
2007    5334
2009    3461
2013    3154
2011    3108
2012    3089
2010    2848
2006    1779
2005     446
2004       4
2003       1
2002       1
2001       1
2000       1
Name: Year, dtype: int64

In [19]:
#Saving our combined data.
cwd_2000_2013_df.to_csv('./Output/2000_2013.csv')

# Load: the final database, tables/collections.

### Connect to local database

In [20]:
rds_connection_string = "root:<insert password>@127.0.0.1/deer_db"
engine = create_engine(f'mysql://{rds_connection_string}')

### Check for tables

In [ ]:
engine.table_names()

### Use pandas to load csv converted DataFrames into database

In [ ]:
cwd_2000_2013_df.to_sql(name='deercwd', con=engine, if_exists='append', index=False)

In [ ]:
census_df.to_sql(name='census', con=engine, if_exists='append', index=False)

### Confirm data has been added by querying the deercwd & census
* NOTE: can also check using pgAdmin

In [ ]:
pd.read_sql_query('select * from deercwd', con=engine).head()

In [ ]:
pd.read_sql_query('select * from census', con=engine).head()